# InitialSetup

In [ ]:
import os
import shutil

if not os.path.exists('data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if not os.path.exists('OUTPUT/'):
    os.makedirs('OUTPUT/')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, AveragePooling2D, MaxPooling2D, Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Loading dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)
training_data = train_gen.flow_from_directory('data_set/train', target_size=(256,256), 
                                                 batch_size=32, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_directory('data_set/val', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory('data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

# Training model from scratch

In [ ]:
classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [ ]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
steps_per_epoch = len(training_data)
validation_steps = len(val_data)

classifier.fit(training_data, epochs=100, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=val_data, validation_steps=validation_steps)

In [ ]:
classifier.save("OUTPUT/ScratchModel.h5")

In [ ]:
_, accuracy = classifier.evaluate(test_data, steps = len(test_data))

In [ ]:
print('ScratchModel accuracy :',accuracy)